In [24]:
# Import the packages needed to generate the samples for simulation parameters and run NorSand triaxial simulations.

import numpy as np
import pandas as pd
import xlwings as xw
import string
from skopt.space import Space
from skopt.sampler import Lhs
import os
import math
import h5py
from scipy.stats import qmc

In [3]:
# dictpos is a dictionary to locate the parameters inside the spreadsheet.
# The dictionaries dict_ranges_ bring the ranges for each input parameter.

dictpos = {"Gamma":[6,4],"lambda":[7,4],"Mtc":[14,4], "N":[15,4],
           "Xtc": [16,4],"H0":[17,4],"Hy":[18,4], "Gmax_p0":[21,4],
           "G_exp": [22,4], "nu":[23,4],"Psi_0":[27,4],"p0":[29,4],
           "K0": [30,4], "OCR": [32,4]}
dict_ranges_material = {"Gamma":[0.9,1.4],"lambda":[0.01,0.07],"Mtc":[1.2,1.5],"N":[0.2,0.5],
           "Xtc": [2,5],"H0":[75,500],"Hy":[200,500], "Gmax_p0":[30,100],
           "G_exp": [0.1,0.6], "nu":[0.1,0.3]}
dict_ranges_test = {"Psi_0":[-0.2,0.2],"p0":[50,1000],
           "K0": [0.8,1.2], "OCR": [0.5,3]}

In [4]:
# Function to change the parameters and run the macros. The inputs are:
# a) final_comp: input parameters as a numpy array of shape (1,14). The parameters need to be inserted
#    in the same order as dictpos.keys(), i.e., ['Gamma', 'lambda', 'Mtc', 'N', 'Xtc', 'H0', 'Hy', 
#    'Gmax_p0', 'G_exp', 'nu', 'Psi_0', 'p0', 'K0', 'OCR'].
# b) dictpos: dictionary to locate the parameters inside the spreadsheet
# c) path_root: path of the spreadsheet "NorTxl.xlsm", obtained at http://www.crcpress.com/product/isbn/9781482213683
# d) type_v: type of the simulation (either "Drained" or "Undrained")
# The function outputs two entities:
# - dictionary containing the parameters inserted to run the simulation;
# - pandas dataframe with simulation results (which are located inside the "Txl SimResults" tab). The columns are:
#   [ep1, epV, p, q, e, pi/p, (pi/p)max, y, Dp, eta].

def run_NorSand(final_comp,dictpos,path_root,type_v):
    letters = list(string.ascii_uppercase)
    wb = xw.Book(path_root)  
    app = wb.app
    macro_vba = app.macro("'NorTxl.xlsm'!RunSim")
    macro_vba_type = app.macro("'NorTxl.xlsm'!ChangeSimMode")
    ws = wb.sheets["Params & Plots"]
    results_comp = []
    for new_v in final_comp:
        for nv,ps in zip(new_v,dictpos.values()):
            pl,pc = ps
            pfinal = letters[pc-1]+str(pl)
            ws[pfinal].value = nv
        if ws["D34"].value == type_v:
            pass
        else:
            macro_vba_type()
        macro_vba()
        ws_results = wb.sheets["Txl SimResults"]
        np_arr = (ws_results['A4'].expand('table')).value                                 
        dd = np.array(np_arr).astype(np.float64)
        dict_inpts = {}
        for keyv,pvalu in zip(dictpos.keys(),new_v.astype(np.float64)):
            dict_inpts[keyv] = pvalu
        dict_inpts["Type"] = type_v
        return dict_inpts,pd.DataFrame(dd)

In [7]:
# Example:
path_root = "D:\\NorSandTXL_H5\\NorTxl.xlsm"
type_v = "Drained"
final_comp = np.array([[1.1,0.04,1.3,0.4,3,200,350,50,0.3,0.2,0.1,500,0.95,1.2]])
a1,a2 = run_NorSand(final_comp,dictpos,path_root,type_v)

In [8]:
a1

{'Gamma': 1.1,
 'lambda': 0.04,
 'Mtc': 1.3,
 'N': 0.4,
 'Xtc': 3.0,
 'H0': 200.0,
 'Hy': 350.0,
 'Gmax_p0': 50.0,
 'G_exp': 0.3,
 'nu': 0.2,
 'Psi_0': 0.1,
 'p0': 500.0,
 'K0': 0.95,
 'OCR': 1.2,
 'Type': 'Drained'}

In [10]:
a2.columns =  ["ep1", "epV", "p", "q", "e", "pi/p", "(pi/p)max", "y", "Dp", "eta"]
a2

,ep1,epV,p,q,e,pi/p,(pi/p)max,y,Dp,eta
0,0.000000,0.000000,500.000000,25.862069,0.951416,0.459374,0.793923,0.100000,0.770230,0.051724
1,0.072409,0.044378,529.847258,113.052071,0.950550,0.433497,0.793923,0.100000,0.770230,0.213367
2,0.085730,0.053719,532.500333,121.011294,0.950367,0.438151,0.791265,0.101453,1.086633,0.227251
3,0.098848,0.062883,535.084441,128.763620,0.950189,0.442688,0.791233,0.101471,1.072749,0.240642
4,0.111772,0.071877,537.603105,136.319610,0.950013,0.447112,0.791206,0.101486,1.059358,0.253569
...,...,...,...,...,...,...,...,...,...,...
3995,19.825467,4.490866,819.705758,982.627570,0.863780,0.925076,0.928499,0.032147,0.101272,1.198756
3996,19.829635,4.491366,819.718809,982.666722,0.863770,0.925096,0.928519,0.032138,0.101244,1.198785
3997,19.833802,4.491865,819.731856,982.705863,0.863761,0.925117,0.928538,0.032129,0.101215,1.198814
3998,19.837969,4.492365,819.744900,982.744994,0.863751,0.925137,0.928558,0.032120,0.101186,1.198842


In [34]:
# This function generates a nested Latin Hypercube Sample for the parameters needed to run the NorSand VBA code. 
# More details can be foun in the paper "NorSand4AI: A Comprehensive Triaxial Test Simulation Database for 
#  NorSand Constitutive Model Materials".

def gen_NorSand_par_2(dict_ranges_material,dict_ranges_test,n_samples,n_samples_2):
    lhs = Lhs(lhs_type="centered", criterion='maximin')
    lhsinner = Lhs(criterion="ratio")
    space_material = Space([(0, 1.) for x in range(len(dict_ranges_material))])
    space_test = Space([(0, 1.) for x in range(len(dict_ranges_test))])
    x_mat = lhs.generate(space_material.dimensions, n_samples,random_state=11)
    data_inp_mat = (np.array(x_mat).T)
    data_expand_mat = []
    for ind_vals in range(len(dict_ranges_material)):
        vlow,vup = list(dict_ranges_material.values())[ind_vals]
        data_pts = data_inp_mat[ind_vals]
        data_expand_mat.append((vup-vlow)*data_pts + vlow)
    data_expand_mat = np.round(np.array(data_expand_mat),4)
    data_expand_tst_corretos=[]
    for pbb,yv in enumerate(data_expand_mat.T):
        x_tst = lhsinner.generate(space_test.dimensions, n_samples_2,random_state=int(11+2*pbb))
        data_inp_tst = (np.array(x_tst).T)
        data_expand_tst = []
        for ind_vals in range(len(dict_ranges_test)):
            if ind_vals==0:
                data_expand_tst.append(data_inp_tst[ind_vals])
            else:
                vlow,vup = list(dict_ranges_test.values())[ind_vals]
                data_pts = data_inp_tst[ind_vals]
                data_expand_tst.append((vup-vlow)*data_pts + vlow)    
        data_expand_tst = np.array(data_expand_tst)
        data_expand_tst_prov = data_expand_tst.copy()
        data_expand_tst_prov[0] = np.array([(np.clip(yv[2]/(yv[4]*(1+yv[3])),0, yv[2]/(5*yv[4]*(1+yv[3])))+0.2)*lhsv-0.2 for lhsv in data_expand_tst_prov[0]])
        data_expand_tst_corretos.append(data_expand_tst_prov)
    data_expand_tst_corretos = np.round(np.array(data_expand_tst_corretos),4)
    final_comp=[]
    for mat_vals,tst_vals in zip(data_expand_mat.T,data_expand_tst_corretos):
        for ti_vals in tst_vals.T:
            final_comp.append(np.concatenate((mat_vals,ti_vals),axis=0))
    return final_comp

In [28]:
# Example
n_samples,n_samples_2 = 128,10

In [29]:
LHS_par = gen_NorSand_par_2(dict_ranges_material,dict_ranges_test,n_samples,n_samples_2)

In [30]:
df = pd.DataFrame(LHS_par)
df.columns = dictpos.keys()
df

,Gamma,lambda,Mtc,N,Xtc,H0,Hy,Gmax_p0,G_exp,nu,Psi_0,p0,K0,OCR
0,1.1988,0.0693,1.4168,0.3371,2.6211,382.1289,367.5781,96.4453,0.5512,0.2148,0.0404,276.7033,1.0608,0.8808
1,1.1988,0.0693,1.4168,0.3371,2.6211,382.1289,367.5781,96.4453,0.5512,0.2148,-0.0926,793.5602,1.1896,1.6783
2,1.1988,0.0693,1.4168,0.3371,2.6211,382.1289,367.5781,96.4453,0.5512,0.2148,-0.0692,470.5439,0.8350,0.5338
3,1.1988,0.0693,1.4168,0.3371,2.6211,382.1289,367.5781,96.4453,0.5512,0.2148,-0.0070,417.0827,1.0035,2.5066
4,1.1988,0.0693,1.4168,0.3371,2.6211,382.1289,367.5781,96.4453,0.5512,0.2148,-0.1249,77.9496,0.9871,2.3146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,1.0074,0.0346,1.3113,0.4098,4.5430,232.7148,205.8594,93.7109,0.1059,0.2773,-0.1004,928.4855,1.0077,2.9350
1276,1.0074,0.0346,1.3113,0.4098,4.5430,232.7148,205.8594,93.7109,0.1059,0.2773,-0.0374,267.8035,0.8080,0.9459
1277,1.0074,0.0346,1.3113,0.4098,4.5430,232.7148,205.8594,93.7109,0.1059,0.2773,-0.1979,781.3645,0.9417,1.2831
1278,1.0074,0.0346,1.3113,0.4098,4.5430,232.7148,205.8594,93.7109,0.1059,0.2773,-0.0247,818.3526,0.8682,2.4294


In [35]:
# This function generates a nested Sobol and Halton sample for the parameters needed to run the NorSand VBA code. 
# More details can be foun in the paper "NorSand4AI: A Comprehensive Triaxial Test Simulation Database for 
#  NorSand Constitutive Model Materials".

def gen_NorSand_par_LD(dict_ranges_material,dict_ranges_test,n_samples,n_samples_2):
    sampler = qmc.Sobol(d=len(dict_ranges_material), scramble=True,seed=11)
    x_mat = sampler.random_base2(m=int(np.log2(n_samples)))
    data_inp_mat = x_mat.T
    data_expand_mat = []
    for ind_vals in range(len(dict_ranges_material)):
        vlow,vup = list(dict_ranges_material.values())[ind_vals]
        data_pts = data_inp_mat[ind_vals]
        data_expand_mat.append((vup-vlow)*data_pts + vlow)
    data_expand_mat = np.round(np.array(data_expand_mat),4)
    data_expand_tst_corretos=[]
    for pbb,yv in enumerate(data_expand_mat.T):
        samplerinner = qmc.Halton(d=len(dict_ranges_test),scramble=True,seed=int(11+2*pbb))
        x_tst = samplerinner.random(n=n_samples_2)
        data_inp_tst = x_tst.T
        data_expand_tst = []
        for ind_vals in range(len(dict_ranges_test)):
            if ind_vals==0:
                data_expand_tst.append(data_inp_tst[ind_vals])
            else:
                vlow,vup = list(dict_ranges_test.values())[ind_vals]
                data_pts = data_inp_tst[ind_vals]
                data_expand_tst.append((vup-vlow)*data_pts + vlow)    
        data_expand_tst = np.array(data_expand_tst)
        data_expand_tst_prov = data_expand_tst.copy()
        data_expand_tst_prov[0] = np.array([(np.clip(yv[2]/(yv[4]*(1+yv[3])),0,yv[2]/(5*yv[4]*(1+yv[3])))+0.2)*lhsv-0.2 for lhsv in data_expand_tst_prov[0]])
        data_expand_tst_corretos.append(data_expand_tst_prov)
    data_expand_tst_corretos = np.round(np.array(data_expand_tst_corretos),4)
    final_comp=[]
    for mat_vals,tst_vals in zip(data_expand_mat.T,data_expand_tst_corretos):
        for ti_vals in tst_vals.T:
            final_comp.append(np.concatenate((mat_vals,ti_vals),axis=0))
    return final_comp

In [31]:
# Example
LD_par = gen_NorSand_par_LD(dict_ranges_material,dict_ranges_test,n_samples,n_samples_2)

In [32]:
df_LD = pd.DataFrame(LD_par)
df_LD.columns = dictpos.keys()
df_LD

,Gamma,lambda,Mtc,N,Xtc,H0,Hy,Gmax_p0,G_exp,nu,Psi_0,p0,K0,OCR
0,1.0767,0.0395,1.3692,0.2804,2.2533,193.4321,417.0348,39.8922,0.5718,0.1312,0.0548,504.3368,0.9458,1.8963
1,1.0767,0.0395,1.3692,0.2804,2.2533,193.4321,417.0348,39.8922,0.5718,0.1312,-0.0927,187.6701,1.1858,1.1820
2,1.0767,0.0395,1.3692,0.2804,2.2533,193.4321,417.0348,39.8922,0.5718,0.1312,-0.0190,821.0034,1.0258,2.2535
3,1.0767,0.0395,1.3692,0.2804,2.2533,193.4321,417.0348,39.8922,0.5718,0.1312,-0.1664,398.7812,1.1058,2.6106
4,1.0767,0.0395,1.3692,0.2804,2.2533,193.4321,417.0348,39.8922,0.5718,0.1312,0.0916,82.1145,0.8658,1.5392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,1.0741,0.0484,1.2952,0.3669,3.1582,116.9126,369.5662,43.8106,0.4111,0.2777,-0.1208,843.3020,1.0031,2.0977
1276,1.0741,0.0484,1.2952,0.3669,3.1582,116.9126,369.5662,43.8106,0.4111,0.2777,-0.0558,421.0797,1.0831,2.8120
1277,1.0741,0.0484,1.2952,0.3669,3.1582,116.9126,369.5662,43.8106,0.4111,0.2777,-0.1858,104.4131,0.9231,1.8426
1278,1.0741,0.0484,1.2952,0.3669,3.1582,116.9126,369.5662,43.8106,0.4111,0.2777,0.0579,737.7464,0.8431,1.4854


In [36]:
# In order to perform the simulation of triaxial tests, the function run_NorSand_simus_P can be called.
# More details can be foun in the paper "NorSand4AI: A Comprehensive Triaxial Test Simulation Database for 
#  NorSand Constitutive Model Materials".

def run_NorSand_simus_P(final_comp,dictpos,n_samples_2,path_xlsm):
    letras = list(string.ascii_uppercase)
    wb = xw.Book(path_xlsm)  
    app = wb.app
    macro_vba = app.macro("'NorTxl.xlsm'!RunSim")
    macro_vba_type = app.macro("'NorTxl.xlsm'!ChangeSimMode")
    ws = wb.sheets["Params & Plots"]
    results_comp = []
    for idini,new_v in enumerate(final_comp):
        matv = int(math.floor(idini/n_samples_2))
        for nv,ps in zip(new_v,dictpos.values()):
            pl,pc = ps
            pfinal = letras[pc-1]+str(pl)
            ws[pfinal].value = nv
        for type_v in ["Drained","Undrained"]:
            if ws["D34"].value == type_v:
                pass
            else:
                macro_vba_type()
            macro_vba()
            ws_results = wb.sheets["Txl SimResults"]
            np_arr = (ws_results['A4'].expand('table')).value 
            path_xlsm_init = ("\\").join(path_xlsm.split("\\")[:-1])
            new_h5_file = path_xlsm_init+'\\Simus\\'+type_v+"\\Par_"+str(matv)+"_"+str(idini)+".h5"
            new_h5_file_spl = new_h5_file.split("\\")
            for va in range(-3,0):
                try:
                    os.mkdir(os.path.join(*new_h5_file_spl[:va]))   
                except:
                    pass
            h5f = h5py.File(new_h5_file, 'w')
            dd = h5f.create_dataset('NorSandTXL', data=np.array((ws_results['A4'].expand('table')).value).astype(np.float32),compression='gzip')
            for keyv,pvalu in zip(dictpos.keys(),new_v.astype(np.float32)):
                dd.attrs[keyv] = pvalu
            dd.attrs["Type"] = type_v
            h5f.close()

In [37]:
# These are two examples of applying the function run_NorSand_simus_P to the input parameters previously generated.
# More details can be foun in the paper "NorSand4AI: A Comprehensive Triaxial Test Simulation Database for 
#  NorSand Constitutive Model Materials".

#run_NorSand_simus_P(LHS_par,dictpos,n_samples_2,path_root)
#run_NorSand_simus_P(LD_par,dictpos,n_samples_2,path_root)